In [ ]:
!pip install transformers datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
data = pd.read_csv('df_data_result_v3.csv')

In [ ]:
data.columns

Index(['sentence_1', 'sentence_2'], dtype='object')

In [ ]:
datav = data[['sentence_1','sentence_2']]

In [ ]:
datav

,sentence_1,sentence_2
0,서울 가산동에서 남대문시장가는 버스노선을 알고싶습니다,가산동 어디에서 출발하십니까?
1,가산동 주민센터입니다.,가산동 주민센터에서 남대문시장으로 가는 버스노선은 505번 버스입니다.
2,어느 정류장에서 타야합니까?,문성초등학교 정류장에서 탑승하시면 됩니다.
3,버스요금은 얼마입니까?,1200원 입니다.
4,서울역에서 서울대학교가는 버스노선을 알고싶습니다.,"서울역에서 서울대학교로 가는 버스노선은 750A, 750B 버스입니다."
...,...,...
84320,급여는 유급으로 처리되었나요?,"아니요, 무급휴가라고 들었어요"
84321,코로나 생활비지원금은 뭐예요?,코로나로 확진되어 입원하거나 자가격리중인 자를 대상으로 경제적으로 도움을 드리는 제...
84322,코로나 생활비지원금 신청하면 어떻게 줘요?,통장으로 입금되어집니다.
84323,코로나 생활비지원금 2인가구는 얼마나 주나요?,"2인가구는 774,700원 지급됩니다."


In [ ]:
datav.to_csv("df_result.csv",index=False)

In [ ]:
data = pd.read_csv('df_result.csv')

In [ ]:
data

,sentence_1,sentence_2
0,서울 가산동에서 남대문시장가는 버스노선을 알고싶습니다,가산동 어디에서 출발하십니까?
1,가산동 주민센터입니다.,가산동 주민센터에서 남대문시장으로 가는 버스노선은 505번 버스입니다.
2,어느정류장에서 타야합니까?,문성초등학교 정류장에서 탑승하시면 됩니다.
3,버스요금은 얼마입니까?,1200원 입니다.
4,서울역에서 서울대학교가는 버스노선을 알고싶습니다.,"서울역에서 서울대학교로 가는 버스노선은 750A, 750B 버스입니다."
...,...,...
85059,"네, 무급휴직 고용유지지원금 지원사업이 있습니다.",그건 뭔가요?
85060,코로나19로 집합금지 및 제한된 기업체에 지원하는 사업을 말합니다.,그럼 우리 독서실도 해당되나요?
85061,네 집합금지 업종으로 지원금 신청에 해당됩니다.,너무나 다행이네요
85062,지원요건이 있을까요?,종로구관내 50인미만 기업체 소속 근로자 중 고용보험 가입자이어야 합니다.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="df_data_result_v3.csv")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b2b426e7e151873e/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset['train']

Dataset({
    features: ['sentence_1', 'sentence_2'],
    num_rows: 85064
})

In [ ]:
books = dataset["train"].train_test_split(test_size=0.2)

In [ ]:
books

DatasetDict({
    train: Dataset({
        features: ['sentence_1', 'sentence_2'],
        num_rows: 68051
    })
    test: Dataset({
        features: ['sentence_1', 'sentence_2'],
        num_rows: 17013
    })
})

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cosmoquester/bart-ko-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
source_lang = "ko"
target_lang = "ko"
prefix = "chatbot: "


def preprocess_function(examples):
    inputs = [prefix + example for example in examples["sentence_1"]]
    targets = [example for example in examples["sentence_2"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/68051 [00:00<?, ? examples/s]

Map:   0%|          | 0/17013 [00:00<?, ? examples/s]

In [ ]:
tokenized_books

DatasetDict({
    train: Dataset({
        features: ['sentence_1', 'sentence_2', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 67460
    })
    test: Dataset({
        features: ['sentence_1', 'sentence_2', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16865
    })
})

In [ ]:
checkpoint = "cosmoquester/bart-ko-base"

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [ ]:
tokenized_books["train"]

Dataset({
    features: ['sentence_1', 'sentence_2', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 68051
})

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained('cosmoquester/bart-ko-base')

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at cosmoquester/bart-ko-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=20)

Epoch 1/20
4253/4253 [==============================] - 999s 235ms/step - loss: 0.9614 - val_loss: 2.0771
Epoch 2/20
4253/4253 [==============================] - 997s 235ms/step - loss: 0.9157 - val_loss: 2.0963
Epoch 3/20
4253/4253 [==============================] - 997s 234ms/step - loss: 0.8727 - val_loss: 2.1076
Epoch 4/20
4253/4253 [==============================] - 997s 234ms/step - loss: 0.8300 - val_loss: 2.1215
Epoch 5/20
4253/4253 [==============================] - 997s 234ms/step - loss: 0.7912 - val_loss: 2.1359
Epoch 6/20
4253/4253 [==============================] - 996s 234ms/step - loss: 0.7545 - val_loss: 2.1557
Epoch 7/20
4253/4253 [==============================] - 996s 234ms/step - loss: 0.7163 - val_loss: 2.1724
Epoch 8/20
4253/4253 [==============================] - 996s 234ms/step - loss: 0.6835 - val_loss: 2.1932
Epoch 9/20
4253/4253 [==============================] - 998s 235ms/step - loss: 0.6518 - val_loss: 2.2086
Epoch 10/20
4253/4253 [=======================

In [ ]:
model.config

BartConfig {
  "_name_or_path": "cosmoquester/bart-ko-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 2,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 3,
  "forced_eos_token_id": 3,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 2048,
  "model_type": "bart",
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "scale_embedding": false,
  "torch_dtype": "float32",

In [1]:
from transformers import AutoTokenizer

Quest = '서울역에서 서울대입구 가는 버스 알려줘'
inputs = tokenizer(Quest, return_tensors="tf").input_ids
outputs = model.generate(inputs, max_new_tokens=10, num_beams=5, do_sample=True, top_k=30, top_p=0.95)
AIanswer = tokenizer.decode(outputs[0], skip_special_tokens=True)
AIanswer

In [ ]:
tf_save_directory = "./tf_model"
tokenizer.save_pretrained(tf_save_directory)
model.save_pretrained(tf_save_directory)

In [1]:
tf_model = TFAutoModelForSeq2SeqLM.from_pretrained(tf_save_directory)

In [ ]:
tf_model.summary()

Model: "tf_bart_for_conditional_generation_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  126965760 
                                                                 
 final_logits_bias (BiasLaye  multiple                 32000     
 r)                                                              
                                                                 
Total params: 126,997,760
Trainable params: 126,965,760
Non-trainable params: 32,000
_________________________________________________________________
